In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the dataset
df_cleaned=pd.read_csv('../data/cleaned_data.csv')

In [2]:
df_cleaned.head()

,index,test_time,cycle_index,cell_index,voltage,discharge_capacity,current,internal_resistance,temperature
0,834355,1801.8327,0.0,2017-05-12_5_4C-50per_3C_CH13,3.291040,0.000016,-0.450999,0.021151,29.973852
1,834356,1811.8329,0.0,2017-05-12_5_4C-50per_3C_CH13,3.281339,0.001570,-0.559847,0.021151,30.012213
2,834357,1821.8363,0.0,2017-05-12_5_4C-50per_3C_CH13,3.277671,0.003125,-0.559859,0.021151,29.995052
3,834358,1831.8373,0.0,2017-05-12_5_4C-50per_3C_CH13,3.275081,0.004680,-0.559828,0.021151,29.981087
4,834359,1841.8468,0.0,2017-05-12_5_4C-50per_3C_CH13,3.273016,0.006237,-0.559821,0.021151,30.000351


In [ ]:
df_cleaned["cell_index"].unique()

### converting all data into numerical form : encoding

In [3]:
mapping = {
    '2017-05-12_5_4C-50per_3C_CH13': 0,
    '2017-05-12_5_4C-50per_3C_CH14': 1,
    '2017-05-12_5_4C-70per_3C_CH17': 2,
    '2017-05-12_4C-80per_4C_CH6': 3,
    '2017-05-12_5_4C-40per_3_6C_CH20': 4,
    '2017-05-12_6C-40per_3_6C_CH33': 5
}

# Replace categorical values with numerical ones
df_cleaned["cell_index"] = df_cleaned["cell_index"].replace(mapping)


C:\Users\Akanksha\AppData\Local\Temp\ipykernel_6920\3379787380.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned["cell_index"] = df_cleaned["cell_index"].replace(mapping)


In [ ]:
df_cleaned


In [ ]:
df_cleaned.info()

In [ ]:
df_cleaned.corr()

In [ ]:
sns.heatmap(df_cleaned.corr(), annot=True)

In [ ]:
sns.heatmap(df_cleaned.corr(), annot=True, cmap="rainbow")

In [ ]:
numerical_features = [feature for feature in df_cleaned.columns if df_cleaned[feature].dtypes != 'O']

In [ ]:
# IQR Method
anomalies_iqr = {}
for feature in numerical_features:
    Q1 = df_cleaned[feature].quantile(0.25)
    Q3 = df_cleaned[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    anomalies_iqr[feature] = df_cleaned[(df_cleaned[feature] < lower_bound) | (df_cleaned[feature] > upper_bound)]



In [ ]:
# Display anomalies detected by IQR
for feature, anomaly_data in anomalies_iqr.items():
    print(f"Anomalies in {feature} (IQR):\n", anomaly_data)

# Handling anamolies

In [ ]:
# Calculate IQR
Q1 = df_cleaned[feature].quantile(0.25)
Q3 = df_cleaned[feature].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out rows outside the bounds
df_no_outliers = df_cleaned[(df_cleaned[feature] >= lower_bound) & (df_cleaned[feature] <= upper_bound)]


In [ ]:
# Replace outliers with median
median = df_cleaned[feature].median()
df_cleaned[feature] = np.where((df_cleaned[feature] < lower_bound) | (df_cleaned[feature] > upper_bound), median, df_cleaned[feature])


In [ ]:
# Add a column to flag outliers
df_cleaned['is_outlier'] = ((df_cleaned[feature] < lower_bound) | (df_cleaned[feature] > upper_bound)).astype(int)


In [ ]:
for feature in numerical_features:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df_cleaned[feature])
    plt.title(f'Boxplot of {feature} (Outliers Removed)')
    plt.show()

In [ ]:
sns.pairplot(df_cleaned)